In [1]:
from imjoy import api
import numpy as np
import os
import Tracker_loli
from pycromanager import Bridge, Acquisition, multi_d_acquisition_events
import matplotlib.pyplot as plt
from tifffile import imsave
from IPython.display import clear_output

from tifffile import imread, imsave
import cv2
bridge = Bridge()
bridge2 = Bridge()
core = bridge.get_core()
core2 = bridge2.get_core()
core.get_version_info()

'MMCore version 10.1.1'

In [2]:
def snap_image():
    # acquire an image and display it
    core.snap_image()
    tagged_image = core.get_tagged_image()
    # get the pixels in numpy array and reshape it according to its height and width
    image_array = np.reshape(
        tagged_image.pix,
        newshape=[-1, tagged_image.tags["Height"], tagged_image.tags["Width"]],
    )
    # for display, we can scale the image into the range of 0~255
    image_array = (image_array / image_array.max() * 255).astype("uint8")
    # return the first channel if multiple exists
    return image_array[0, :, :]

def move_xystage(x,y):
    core.set_xy_position(x,y)
    
def light_on():
    core2.set_property("Arduino-Switch", "State", "1")
    core2.set_property("Arduino-Shutter", "OnOff", "1")

def light_off():
    core2.set_property("Arduino-Switch", "State", "1")
    core2.set_property("Arduino-Shutter", "OnOff", "0")

def set_Exposure(x):
    exp = core.get_exposure()
    print("Changing exposure from" + str(exp) + " to" + str(x))
    core.set_exposure(x)
    
def save_img(path,name, img):
    imsave(os.path.join(path,name), img)

def get_XY_position():
    x = core.get_x_position()
    y = core.get_y_position()
    return x,y

def change_FilterWheel(f):
    f = int(f)
    if f < 0 or f > 10:
        print("Filter must be integer between 0 and 10")
    else:
        psrint("Changing from filter " + core.get_property("FilterWheel", "State") + " to " + str(f))
        core.set_property("FilterWheel", "State", str(f))

def take_stack(n, fn, save=True):
    img = snap_image()
    stack = np.zeros((n,img.shape[0],img.shape[1]))
    for i in range(n):
        stack[i,:,:] = snap_image()
    
    if save:
        stack = np.cast["uint16"](stack)
        imsave(fn, stack)
    return stack

In [3]:
path = r"C:\Users\lmorales-curiel\Desktop"
name = "a"
stack = take_stack(10, fn = os.path.join(path, name+".tif"), save=True)
box = Tracker_loli.Tracker(path=path, filename=name, img = stack, initBB = None)
x,y = get_XY_position()
print(x,y,box)

sx = box[0] - 250
x -= sx

sy = box[1] - 250
y -= sy

x += box[0]
y += box[1]
print(x,y)
move_xystage(x, y)
imsave()

0
<class 'tuple'>
True (149, 224, 263, 206)
2
True (148, 224, 263, 206)
3
True (149, 224, 263, 206)
4
True (150, 224, 263, 206)
5
True (151, 226, 263, 206)
6
True (150, 224, 263, 206)
7
True (151, 226, 263, 206)
8
True (150, 224, 263, 206)
9
True (151, 224, 263, 206)
10
-8487.400390625 530.7999877929688 (151, 224, 263, 206)
-8237.400390625 780.7999877929688


In [4]:
while True:
    stack = take_stack(10, fn = os.path.join(path, name+".tif"), save=True)
    box = Tracker_loli.Tracker(path, name, img = stack, initBB = box)
    x,y = get_XY_position()
    sx = box[0] - 250
    x -= sx

    sy = box[1] - 250
    y -= sy
    move_xystage(x, y)

True (151, 224, 263, 206)
0
True (156, 222, 263, 206)
1
True (167, 218, 263, 206)
2
True (180, 214, 263, 206)
3
True (193, 210, 263, 206)
4
True (204, 206, 263, 206)
5
True (215, 202, 263, 206)
6
True (224, 200, 263, 206)
7
True (233, 198, 263, 206)
8
True (238, 192, 262, 206)
9
True (238, 192, 262, 206)
0
True (244, 190, 256, 206)
1
True (252, 188, 248, 206)
2
True (258, 186, 242, 206)
3
True (262, 184, 238, 206)
4
True (266, 182, 234, 206)
5
True (270, 180, 230, 206)
6
True (274, 178, 226, 206)
7
True (278, 176, 222, 206)
8
True (284, 174, 216, 206)
9
True (284, 174, 216, 206)
0
True (288, 172, 212, 206)
1
True (290, 172, 210, 206)
2
True (294, 172, 206, 206)
3
True (296, 172, 204, 206)
4
True (300, 170, 200, 206)
5
True (302, 168, 198, 206)
6
True (304, 168, 196, 206)
7
True (306, 166, 194, 206)
8
True (308, 164, 192, 206)
9
True (308, 164, 192, 206)
0
True (310, 162, 190, 206)
1
True (312, 158, 188, 206)
2
True (312, 156, 188, 206)
3
True (312, 156, 188, 206)
4
True (314, 156, 186,

KeyboardInterrupt: 